In [1]:
import pandas as pd
import sqlite3

## создаем подключение к базе данных с помощью библиотеки sqlite3

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite', check_same_thread=False)

## получить схему таблицы теста

In [3]:
pd.read_sql('PRAGMA table_info(test)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## получить только первые 10 строк таблицы, чтобы проверить, как выглядит таблица

In [4]:
pd.read_sql('select * from test limit 10', conn)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## найти среди всех пользователей минимальное значение дельты между первым коммитом пользователя и крайним сроком соответствующей лаборатории, используя только один запрос

In [5]:
query = """
SELECT uid, MIN(cast((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INT)) AS min_diff
FROM test t
LEFT JOIN deadlines dl on t.labname = dl.labs
WHERE NOT t.labname = 'project1'
ORDER BY 2
LIMIT 1
"""

df_min = pd.read_sql(query, conn)
df_min

,uid,min_diff
0,user_30,-202


## сделать то же самое, но для максимума, используя только один запрос, имя фрейма данных — df_max

In [6]:
query = """
SELECT uid, MAX(cast((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INT)) AS max_diff
FROM test t
LEFT JOIN deadlines dl on t.labname = dl.labs
WHERE NOT t.labname = 'project1'
ORDER BY 2
LIMIT 1
"""

df_max = pd.read_sql(query, conn)
df_max

,uid,max_diff
0,user_25,-2


## сделайте то же самое, но для среднего значения, используя только один запрос, на этот раз ваш фрейм данных не должен включать столбец uid, а имя фрейма данных — df_avg

In [7]:
query = """
SELECT AVG(cast((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INT)) AS avg_diff
FROM test t
LEFT JOIN deadlines dl on t.labname = dl.labs
WHERE NOT t.labname = 'project1'
"""

df_avg = pd.read_sql(query, conn)
df_avg

,avg_diff
0,-89.125


## Мы хотим проверить гипотезу о том, что пользователи, которые посетили ленту новостей всего несколько раз, имеют меньшую разницу между первым коммитом и дедлайном. Для этого необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей.

In [8]:
query = """
SELECT
    t.uid,
    (CAST((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS INT)) AS avg_diff,
    COUNT(DISTINCT pageviews.datetime) as pageviews
FROM
    test t
LEFT JOIN 
    deadlines d ON t.labname = d.labs
LEFT JOIN
    pageviews ON t.uid = pageviews.uid
WHERE 
    t.labname != 'project1'
GROUP BY
    t.uid
"""
views_diff = pd.read_sql(query, conn)

views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.062967
pageviews,-0.062967,1.000000


## закрыть соединение

In [9]:
conn.close()